# Code for image cropping and perspective transform
The code looks for 2 rectangular shapes in the image (see figure below), cropes them and eventually crops from them a specified area of interest (see below). Ratio of sizes of the 2 devices need to be provided. Can be easy adjust to also work with just one device (or another number of devices).


1|2|3|4|5
-|-|-|-|- 
<img src="im46.jpg" alt="Drawing" width="420"/> | <img src="small_im46.jpg" alt="Drawing" width="150"/> | <img src="im46 2.jpg" alt="Drawing" width="150"/> | <img src="AOIsmall_im46.jpg" alt="Drawing" width="50"/>| <img src="AOIim46.jpg" alt="Drawing" width="50"/>

The sequence of work of the script. In yellow frame are the 2 devices, in blue are the areas of interest (AOI)

In [7]:
# import the necessary packages
#from pyimagesearch.transform import four_point_transform
#from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
from shutil import copyfile

import os

### User selected input values
The first part needs user change, the second part can probably be kept with the default values

In [43]:
###### the following features need to be changed by the user to desired values

# file path to the input images, that will be cropped by the script
input_images_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/Val_AHRI_Test_images/'
# output folder where the intermediate cropped images will be stored
output_cropped_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/cropped/'
# output folder where the images will be stored in which the squares could not be successfully detected
output_uncropped_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/uncropped/'
# output folder where eventually the defined areas of interest from the images will be stored
output_AOI_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/AOI/'

# define the coordinates of your areas of interest, as height and width ratio in respect to the 4 squares.
# The coordinates will have the form of: 
# image_AOI = im[int(x0*height_im):int(x1*height_im), int(y0*width_im):int(y1*width_im)]
# In this script 2 devixes are being detected, L is the large and S is the small

L_top, L_bottom = 0.44, 0.65
L_left, L_right = 0.37, 0.67
S_top, S_bottom = 0.28, 0.63
S_left, S_right = 0.32, 0.66

####### the features below this line can be probably kept with the given default values,
####### but the user may consider changing them if script performance is bad

# what type of image files do you want the script to opearate on
file_extensions = ['.jpg', '.png']

# # the size of the kernel when threshholding is being performed
# thresh_size = 15

# the height of the image will be scaled down to an image with this height, so that working can go faster 
height_working_image=500.

# defining the range of blurness in which the image will be looped for the detection of shapes
GBlur_bottom_value=7
GBlur_top_value=21
GBlur_interval=2

# are_ratio determines the approximate ratio (pixel-area small device)/(pixel-area large device)
# the lower and top biundaries determine the tolerance range around the area_ratio, e.g.
# for a object to be considered small device it has to fulfill 
# area_small>lower_area_boundary*area_ratio*area_big and area_small<top_area_boundary*area_ratio*area_big
area_ratio=1/1.46
lower_area_boundary, top_area_boundary = 0.7, 1.3

#define the range of tolerance for the ratio of h/w for the size of the large device
h_w_ratio_large_lower=2.6
h_w_ratio_large_upper=3.5

# the size of the blurring kernel for the function for correct orientation
blur_size_correct_orientation = 21

In [44]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
 
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
 
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
 
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
 
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
 
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
 
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
 
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
	# return the warped image
	return warped

In [45]:
def width_calc(quader):
    '''Function that assumes quader is rectanlge and returns its width'''
    quad = np.resize(quader, (4,2))
    distances = []
    for coo in quad[1:]:
        d = np.sqrt((quad[0][0]-coo[0])**2 + (quad[0][1]-coo[1])**2)
        distances.append(d)

    return(min(distances))

In [46]:
def main(input_folder, output_folder, output_uncrop_folder, file_extensions, 
         height_working_image, area_ratio, h_w_ratio_large_upper, h_w_ratio_large_lower):
    """Main function, contains the loop and all the sub-functions in it.
    DON'T forget to add an extra '/' at the end of input and output folder.
    area_ratio is the ratio (imaged area small device)/(imaged area large device).
    h_w_ratio_large_upper and h_w_ratio_large_lower are the range margins for the ratio h/w between the lenght and width of the imaged large device."""
    # create a list of all the image-paths in a given folder

    file_names = [fn for fn in sorted(os.listdir(input_folder))
                  if any(fn.endswith(ext) for ext in file_extensions)]
    paths = [input_folder+file_name for file_name in file_names]
    
    flag_L = 'stay in loop'
    flag_S = 'stay in loop'
    i = 0
                
    # loop through all of the image-paths
    for im_path in paths[:]:
        print('loop:', i)
        
        if i>0 and flag_L == 'stay in loop':
            copyfile(paths[i-1], output_uncrop_folder+'large_uncr'+path_end)
        if i>0 and flag_S == 'stay in loop':
            copyfile(paths[i-1], output_uncrop_folder+'small_uncr'+path_end)
        
        flag_L = 'stay in loop'
        flag_S = 'stay in loop'
        i+=1
        
        # loop through all the possible sizes of the Gaussian-Blur-kernel
        for GBlur_size in range(GBlur_bottom_value,GBlur_top_value,GBlur_interval):
#                 print('->GBlur =', GBlur_size)
            if flag_L == 'move to next image':
                continue
#                 # loop through all the possible canny thresholds
#                 for canny_th_min, canny_th_max in zip(low_list, high_list):
#                     if flag == 'move to next image':
#                         continue


            # load the image and compute the ratio of the old height
            # to the new height, clone it, and resize it
            #image = cv2.imread(args["image"])
            image = cv2.imread(im_path)
            ratio = image.shape[0] /height_working_image
            orig = image.copy()
            image = imutils.resize(image, height = int(height_working_image))
            
#             cv2.imshow("Color", image)
#             cv2.waitKey(0)


            # convert the image to grayscale, blur it, and find edges
            # in the image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (GBlur_size, GBlur_size), 0)
            # line below was not in original script, put by Nestor
#                 gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,adapThresh_size,2)

            ########
            # Calculate the canny-edged image based on the median value of image
            v = np.median(gray)
            sigma = 0.33

            # apply automatic Canny edge detection using the computed median
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
            edged = cv2.Canny(gray, lower, upper)
            ########

            # iterating over increasing and decreasing the width of the edges to get rid of gaps and noisy fine structure
            edged = cv2.dilate(edged, None, iterations=1)
            edged = cv2.erode(edged, None, iterations=1)

#             cv2.imshow("Edged", edged)
#             cv2.waitKey(0)

            # find the contours in the edged image, keeping only the
            # largest ones, and initialize the screen contour
            cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

            # loop over the contours
            try:
                for c in cnts:
                    # approximate the contour
                    peri = cv2.arcLength(c, True)
                    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

                    # if our approximated contour has four points, then we
                    # can assume that we have found our screen
                    if len(approx) == 4:
                        warped = four_point_transform(image, approx.reshape(4, 2)) # first version used orig, instead of image. And the there was also a ratio-factor, which I avoided
                        path_end = os.path.basename(im_path)

                        # shape is set to (500,375)
                        h, w = float(max(warped.shape[0],warped.shape[1])), float(min(warped.shape[0],warped.shape[1]))
                        if h/w>h_w_ratio_large_lower and h/w<h_w_ratio_large_upper and h<0.9*image.shape[0] and h>50 and w>50:
                            # repeat the crop, but in the original image in original size and save this crop
                            warped_large_dim = four_point_transform(orig, approx.reshape(4, 2)*ratio)
                            cv2.imwrite(output_folder+path_end, warped_large_dim)
                            flag_L = 'move to next image'
                            
        
                            # once the big test device is found loop throught the contours again and try to find the smaller test device
                            area_big = cv2.contourArea(approx)

                            for cnt in cnts:
                                peri = cv2.arcLength(cnt, True)
                                # the epsilon value below is changed to 0.08 from 0.02 below, because we want it to approximate
                                # more roughly to a quader. Look up the docs to cv2.approxPolyDP
                                approx_s = cv2.approxPolyDP(cnt, 0.08 * peri, True)
                                area_small = cv2.contourArea(approx_s)
#                                     if len(approx_s)==4 and area_small>0.85*area_big/1.46:# and area_small<1.15*factor*area_big and width_calc(approx_s)>0.85*np.sqrt(factor)*w and width_calc(approx_s)<1.15*np.sqrt(factor)*w:
#                                         print('SMALL LOOP2')
                                if len(approx_s)==4 and area_small>lower_area_boundary*area_ratio*area_big and area_small<top_area_boundary*area_ratio*area_big and width_calc(approx_s)>lower_area_boundary*np.sqrt(area_ratio)*w and width_calc(approx_s)<top_area_boundary*np.sqrt(area_ratio)*w:

#                                     cv2.drawContours(image, [approx_s], -1, (0, 255, 0), 2)
#                                     cv2.imshow("Outline", image)
#                                     cv2.waitKey(0)
                                    warped_s = four_point_transform(orig, approx_s.reshape(4, 2)*ratio)
                                    cv2.imwrite(output_folder+'small_'+path_end, warped_s)
                                    flag_S = 'move to next image' 
                            break

            except Exception as e:
                print(e)

In [47]:
import numpy as np
import argparse
import cv2
import imutils
import os
from shutil import copyfile
import matplotlib.pyplot as plt
import numpy as np

def correct_orientation_RDTs(input_folder, output_folder, file_extensions, blur_size,
                            L_top, L_bottom, L_left, L_right,
                            S_top, S_bottom, S_left, S_right):
    '''Function that corerctly orients all the images in the given folder to be top up. Function calculates the brightness of the ends 
    of the RDTs and based on that makes a decision where is top and down. First, function for portrait turning needs to be called.
    L_top, S_top a.s.o provide the relative margins of the cropped areas of interest (AIO)'''
    
    file_names = [fn for fn in sorted(os.listdir(input_folder))
                  if any(fn.endswith(ext) for ext in file_extensions)]
    paths = [input_folder+file_name for file_name in file_names]
    
    for im_path in paths[:]:
        im = cv2.imread(im_path)
        # rotate the images, if the second dimension is larger
        if im.shape[1]>im.shape[0]:
            im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)
        # select only the blue channel, there is the greatrst contrast between the red blood dot and the other end of the stripe
        im_blue = im[:,:,0]
        # blur the image slightly to get rid of outlying dark and bright pixels compromising the calculations.
        # Uses blur_size for the size of blurred kernels
        im_blue_blur = cv2.GaussianBlur(im_blue, (blur_size, blur_size), 0)
        # get the height (=500) and width of the image 
        h, w = im_blue_blur.shape[0], im_blue_blur.shape[1]
        # split the image into 2 parts, centered around the possible location of the red blood dot
        im_part1 = im_blue_blur[int(0.1*h):int(0.3*h),int(0.3*w):-int(0.3*w)]
        im_part2 = im_blue_blur[-int(0.3*h):-int(0.1*h),int(0.3*w):-int(0.3*w)]
        
        # check if the second part of the image has a larger gradient (the red spot) and rotate the image if so
        if (np.max(im_part1)-np.min(im_part1))/np.mean(im_part1) > (np.max(im_part2)-np.min(im_part2))/np.mean(im_part2):
            im = cv2.rotate(im, cv2.ROTATE_180)
        
        path_end = os.path.basename(im_path)
        
        # here the function differentiates between the images of different devices,
        # becasue these different devices would have different dimensions, that are referenced for cropping
        # In the lines below we differentiate between 2 different devices, one 'small' and one that isn't, but one could add more if-clauses for differentiations
        # L_top, S_top a.s.o provide the relative margins of the cropped areas of interest (AIO)
        if 'small' in im_path:
            im_AOI = im[int(S_top*h):int(S_bottom*h), int(S_left*w):int(S_right*w)]
            cv2.imwrite(output_folder+'AOI'+path_end, im_AOI)
        else:
            im_AOI = im[int(L_top*h):int(L_bottom*h), int(L_left*w):int(L_right*w)]
            cv2.imwrite(output_folder+'AOI'+path_end, im_AOI)
        
#         cv2.imshow("Edged1", im_AOI)
#         cv2.imshow("Edged2", im_part2)
#         cv2.waitKey(0)

In [48]:
main(input_images_folder, output_cropped_folder, output_uncropped_folder,
     ['.jpg', '.png'],
    height_working_image=height_working_image, area_ratio=1/1.46, h_w_ratio_large_lower=2.6, h_w_ratio_large_upper=3.5)

correct_orientation_RDTs(output_cropped_folder, output_AOI_folder, 
                         ['.png', '.jpg'], blur_size=blur_size_correct_orientation, 
                         L_top=L_top, L_bottom=L_bottom, L_left=L_left, L_right=L_right,
                         S_top=S_top, S_bottom=S_bottom, S_left=S_left, S_right=S_right)

loop: 0
loop: 1
loop: 2
loop: 3
loop: 4
loop: 5
loop: 6
loop: 7
loop: 8
loop: 9
loop: 10
loop: 11
loop: 12
loop: 13
loop: 14
loop: 15
loop: 16
loop: 17
loop: 18
loop: 19
loop: 20
loop: 21
loop: 22
loop: 23
loop: 24
loop: 25
loop: 26
loop: 27
loop: 28
loop: 29
loop: 30
loop: 31
loop: 32
loop: 33
loop: 34
loop: 35
loop: 36
loop: 37
loop: 38
loop: 39
loop: 40
loop: 41
loop: 42
loop: 43
loop: 44
loop: 45
loop: 46
loop: 47
loop: 48
loop: 49
loop: 50
loop: 51
loop: 52
loop: 53
loop: 54
loop: 55
loop: 56
loop: 57
loop: 58
loop: 59
loop: 60
loop: 61
loop: 62
loop: 63
loop: 64
loop: 65
loop: 66
loop: 67
loop: 68
loop: 69
loop: 70
loop: 71
loop: 72
loop: 73
loop: 74
loop: 75
loop: 76
loop: 77
loop: 78
loop: 79
loop: 80
loop: 81
loop: 82
loop: 83
loop: 84
loop: 85
loop: 86
loop: 87
loop: 88
loop: 89
loop: 90
loop: 91
loop: 92
loop: 93
loop: 94
loop: 95
loop: 96
loop: 97
loop: 98
loop: 99


### Remarks regarding the performance of the code:
The code doesn't find the contours of the paper-square if:
- there is a grass going over square
- a shadow is being thrown by the deice itself on the paper-square
    - does reducing the blur help? Or maybe increasing it??

- pictures of each step in the program
- check how saving the coordinates from the pictures might be possible


- ask for actual test pictures
- adjust the contrast so that it can crop around our aoi

2nd idea: try crosssection

### Next things to do:
- Instead of ratio height/width do by pixel size
- Use the threshold function again, maybe
- Try to reach 80% accuracy
- afterwards, check if images have correct orientation (e.g. by checking what is the brightness ratio between the two sides of the image)
- take a cross section through the image and check if cross section fulfills a filter criterium
- crop around area of interest. Just hard code the geometrical coordinates of the stripes section

### New idea:
- instead of detecting the big square, detect the area of interest where the stripes are
- requires to male adjustments to the thresholding and canny
- else, should be easy to implement, just need to filter by pixel size and ratio


- show Val both options:
        - dilate applied on the whole platter
        - dilate applied only on the big test
        - result, 05.02.: 93/100 images cropped, 81/93 are correct test, 1/93 is complete black
        - with blur of (7,7) the correct tests increases to 89/93
        - blur (9,9): 90/92
        - best performance: progressive blur-kernel size, starting at 9, no threshold: 99/100 cropped, 96/99 correct test, 1/99 is completely black
        - further tinkering: change dilate iterations, change threshold to Gaussian instead of adaptive
        
- 06.02: with factoring between 07-1.3 works optimal, for 0.6-1.4 there are false positiivs coming from the big test device. Check in the images to see why some are being cropped correct, what is wrong with their particular contours
- 08.02.: The circle detection is not a great criterium for orientating the image, because sometimes the image is poorly cropped, the cirle is distorted and not properly detected by the algoeithm. Furthermore it would only work in the small device case, for the larger device the opening is not really round - blurr helps for the detection of a circle in the smaller device type, canny gives sometimes errors.
    - check if there would be better cropping if the image would be resized to length 1000 instead of 500 - yes, it is.
    - maybe ignore all images where the circle can not be detected. This should reduce the pool by further 10-20%
    - instead of detecting circle count the number of contours in the top 25% of the image and in the bottom 25% of the image. Count them and where they are =3, this must be the top (with the label "HIV")
    
- increase the height of all images from 500 to 1000. Why is this making the small devices less detectable?

- 15.02.: Orientation fails for one of the pictures. Check why - problem fixed

Next:
- +crop out all areas of interest
- +make all the hardcoded imperical variables input that you can choose
- +sort the issur with pictures that are not cropped not being saved in the designated folder

Next:
- prevent small device to be saved twice
- implement initial cropping by the 4 small squares, not by the shapes of the devices